In [1]:
import numpy as np 
import pandas as pd

In [2]:
df_train = pd.read_csv('./new_data/train_accident_normalized_text.csv')
df_test = pd.read_csv('./new_data/test_accident_normalized_text.csv')
df_combine = pd.read_csv('./new_data/combine_accident_normalized_text.csv')

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
#Vectorization

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_train['text_normalized'])
y_train = df_train['target']
X_test = vectorizer.transform(df_test['text_normalized'])
y_test = df_test['target']

In [7]:
# Определение класса модели LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [8]:
# Определение класса Dataset для загрузки данных
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [9]:
# Подготовка данных и загрузка в DataLoader
X_train_dense = X_train.toarray()
df_train_dataset = CustomDataset(X_train_dense, y_train)
df_train_loader = DataLoader(df_train_dataset, batch_size=64, shuffle=True)

In [10]:
# Определение параметров модели и оптимизатора
input_size = len(X_train_dense[0])
hidden_size = 100
num_layers = 1
output_size = 1
num_epochs = 10
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# Создание экземпляра модели и оптимизатора
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
batch_size = 64

In [13]:
# Обучение модели
for epoch in range(num_epochs):
    for inputs, labels in df_train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Приведение входных данных к правильной размерности
        inputs = inputs.reshape(batch_size, num_layers, input_size)

        # Передача входных данных в модель
        outputs = model(inputs)

        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

RuntimeError: mat1 and mat2 must have the same dtype

In [ ]:
# Оценка модели на тестовых данных
df_test_dataset = CustomDataset(X_test, y_test)
df_test_loader = DataLoader(df_test_dataset, batch_size=64, shuffle=False)

model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        predicted = torch.round(torch.sigmoid(outputs))
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print(f'Test Accuracy: {accuracy * 100:.2f}%')
